<a href="https://colab.research.google.com/github/reagenhuskey/cs290/blob/main/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [4]:
X = pd.read_csv("https://github.com/reagenhuskey/cs290/raw/main/2022%20US%20College%20Rankings.csv")

In [20]:
X.columns

Index(['College Name', 'Adjusted Rank', 'Tuition', 'Enrollment Numbers'], dtype='object')

In [24]:
affordability = []
cheap = 15000 #consider these are mostly private/really good schools, so their tuition is way more expensive than the average state school.
affordable = 30000
expensive = 100000


In [27]:
def categorize_t(t):
  if t <= cheap:
      affordability.append('cheap')
      return 'cheap'
  elif t <= affordable:
      affordability.append('affordable')
      return 'affordable'
  elif t <= expensive:
      affordability.append('expensive')
      return 'expensive'

y = pd.Series(affordability)
X["Tuition Category"] = X["Tuition"].apply(categorize_t)


**unseen instance**

In [22]:
Sewanee = {
    "College Name": "Sewanee",
    "Adjusted Rank": 100, #sounds good enough
    "Tuition Category": "expensive"
}

In [30]:
def find_prior(y):
  classes = y.unique()
  prior = {}
  for c in classes:
    prior[c] = len(y[y == c]) / len(y)
  return prior

In [ ]:
def find_likelihood(x, y):
  likelihood = {}
  for f in X.columns:
    likelihood[f] = {}
    for c in y.unique():

